# Import Requirements

In [1]:
from __future__ import division
import sys
import pysam
import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
import time
from collections import defaultdict
import csv
from itertools import islice
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sys.path.append('/Users/greg/damlarces/')
from damlarces.NGSTools import ReadTools

# How to read in filepaths

In [3]:
files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/BAMs/*.bam')
f = files[0]
print len(files)

96


# Process all Tat 1 files

In [4]:
files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/BAMs/*.bam')
for f in files:
    patient = f.split('/')[-1].split('.')[0].split('-')[0]
    visit = f.split('/')[-1].split('.')[0].split('-')[1]

In [5]:
files = glob.glob('/Users/greg/Desktop/FullNeuroIllumina/BAMs/*.bam')

meta_data = {'Patient':[], 'Visit':[], 'total_reads':[], 'right_length':[], 'no_stop':[]}

for f in files:
    patient = f.split('/')[-1].split('.')[0].split('-')[0]
    visit = f.split('/')[-1].split('.')[0].split('-')[1]
    print patient, visit

    samfile = pysam.Samfile(f, mode='rb').fetch(region='HXB2:5800-6060')

    # time
    t = time.time()

    counts = defaultdict(int)
    it = ReadTools.SeqReadRecord.translate_from_stream(samfile, [('Tat1', 5829, 6046)])
    total_reads=0
    right_length=0
    no_stop=0

    all_len = []
    for orf in it: 
        # count total reads
        total_reads += 1 
        # verify the correct ORF length
        if len(orf.seq)==72:
            right_length += 1
            # check that there is not a premature stop codon
            if '*' not in orf.seq:
                no_stop+=1
                # iterate through the amino acid sequences
                for i, aa in enumerate(orf.seq):
                    pos = i+1
                    # ignore the padded gaps
                    if aa != '-':
                        counts[(patient, visit, orf.chrom, pos, aa)] += 1

    # time
    elapsed = time.time() - t
    print 'Time:', elapsed

    meta_data['Patient'].append(patient)
    meta_data['Visit'].append(visit)
    meta_data['total_reads'].append(total_reads)
    meta_data['right_length'].append(right_length)
    meta_data['no_stop'].append(no_stop)
    
    out_file = f.replace('hivsorted.bam', 'AAcounts.csv')
    out_file = out_file.replace('/FullNeuroIllumina/BAMs', '/FullNeuroIllumina/AAcounts')
    print out_file
    
    df = pd.Series(counts)
    df.index.names = ['Patient', 'Visit', 'ORF', 'Position', 'AA']
    df = df.reset_index()
    df.rename(columns={0:'Count'}, inplace=True)
    df.to_csv(out_file, index=False)

A0001 R09
Time: 164.442930937
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0001-R09.AAcounts.csv
A0002 R11
Time: 0.341635942459
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0002-R11.AAcounts.csv
A0004 R10
Time: 0.108597993851
/Users/greg/Desktop/FullNeuroIllumina/AAcounts/A0004-R10.AAcounts.csv


ValueError: Length of new names must be 1, got 5

In [8]:
pd.DataFrame(meta_data)

,Patient,Visit,no_stop,right_length,total_reads
0,A0001,R09,37923,110987,111132
1,A0002,R11,28,119,119
2,A0004,R10,0,0,0


# Dictionary to dataframe

In [ ]:
out_file = f.replace('hivsorted.bam', 'AAcounts.csv')
out_file = out_file.replace('/FullNeuroIllumina/BAMs', '/FullNeuroIllumina/AAcounts')
print out_file

df = pd.Series(counts)
df.index.names = ['Patient', 'Visit', 'ORF', 'Position', 'AA']
df = df.reset_index()
df.rename(columns={0:'Count'}, inplace=True)

df.to_csv(out_file, index=False)
df.head()

In [ ]:
pcts = df.groupby('Position')
pcts

In [ ]:
df2 = df.groupby(['Position','AA'])['Count'].agg({'Count': 'sum'})
df3 = df2.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))
df3['Count']

In [ ]:
for pos in df.groupby('Position'):
    print pos[0], sum(pos[1]['Count'])